# **Data Collection Notebook**

**Objective:**
Collect and preprocess raw data related to insider transactions and stock prices.
Fetch data from Kaggle and save it as raw data, inspect and save it.

**Inputs:**
- Raw TSV files: `NONDERIV_TRANS.tsv`, `SUBMISSION.tsv`, `REPORTING_OWNER.tsv`
- Stock price data files in `../data/raw/stock_prices/` directory
- Kaggle JSON file - the authentication token.

**Outputs:**
- Interim CSV files:
  - `interim_insider_transactions.csv`
  - `interim_stock_prices.csv`
  - `interim_merged_insider_transactions_stock_prices.csv`

---


# Step 1: Imports & Kaggle Endpoint 

In [1]:
import sys
import json
from pathlib import Path
import os
import pandas as pd
import numpy as np
import re
from dotenv import load_dotenv
import zipfile
import shutil

# Change working directory

* Access current directory and change to parent directory

In [2]:
current_dir = os.getcwd()
current_dir

os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm Current Directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\sawaomar\\Documents\\Project 5\\Repo\\MarketPulseAnalytics'

---

# Fetch data from Kaggle

### Setup Credintials

In [4]:
# Define the path to the .env file in the current directory (MarketPulseAnalytics)
env_path = os.path.join(os.getcwd(), '.env')

# Load environment variables from the .env file if it exists
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(".env file loaded from:", env_path)
else:
    print("No .env file found in the current directory. Ensure environment variables are set in the hosting environment.")

# Access the environment variables
kaggle_username = os.getenv('KAGGLE_USERNAME')
kaggle_key = os.getenv('KAGGLE_KEY')

# Verify environment variables are set
if not kaggle_username or not kaggle_key:
    print("Warning: KAGGLE_USERNAME and/or KAGGLE_KEY environment variables are not set. Make sure they are configured in the production environment.")
else:
    print("Environment variables loaded successfully.")


.env file loaded from: c:\Users\sawaomar\Documents\Project 5\Repo\MarketPulseAnalytics\.env
Environment variables loaded successfully.


Set Kaggle directory

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

Set the download paths

In [6]:
KaggleDatasetPath = "borismarjanovic/price-volume-data-for-all-us-stocks-etfs"
DestinationFolder = "data/raw/stock_prices/downloaded"

Download the dataset

In [7]:
!kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}


  0%|          | 0.00/492M [00:00<?, ?B/s]
  0%|          | 1.00M/492M [00:00<05:24, 1.59MB/s]
  0%|          | 2.00M/492M [00:00<03:09, 2.71MB/s]
  1%|          | 3.00M/492M [00:01<02:28, 3.45MB/s]
  1%|          | 4.00M/492M [00:01<02:10, 3.92MB/s]
  1%|          | 5.00M/492M [00:01<01:58, 4.31MB/s]
  1%|          | 6.00M/492M [00:01<01:52, 4.53MB/s]
  1%|▏         | 7.00M/492M [00:01<01:47, 4.74MB/s]
  2%|▏         | 8.00M/492M [00:02<01:48, 4.66MB/s]
  2%|▏         | 9.00M/492M [00:02<01:44, 4.84MB/s]
  2%|▏         | 10.0M/492M [00:02<01:43, 4.90MB/s]
  2%|▏         | 11.0M/492M [00:02<01:40, 4.99MB/s]
  2%|▏         | 12.0M/492M [00:02<01:39, 5.06MB/s]
  3%|▎         | 13.0M/492M [00:03<01:38, 5.12MB/s]
  3%|▎         | 14.0M/492M [00:03<01:37, 5.14MB/s]
  3%|▎         | 15.0M/492M [00:03<01:36, 5.15MB/s]
  3%|▎         | 16.0M/492M [00:03<01:36, 5.17MB/s]
  3%|▎         | 17.0M/492M [00:03<01:37, 5.11MB/s]
  4%|▎         | 18.0M/492M [00:04<01:36, 5.12MB/s]
  4%|▍         | 19.

Unzip the downloaded file, and delete the zip file
Define dataset, destination folder and download.
Clean up.

In [8]:
# Path to the downloaded ZIP file
zip_file_path = os.path.join(DestinationFolder, "price-volume-data-for-all-us-stocks-etfs.zip")

# Unzip the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

# Define paths for ETFs folder, Stocks folder, and target folder
etfs_folder = os.path.join(DestinationFolder, "ETFs")
stocks_folder = os.path.join(DestinationFolder, "Stocks")
target_folder = os.path.join("data", "raw", "stock_prices")

# Delete the ETFs folder if it exists
if os.path.exists(etfs_folder):
    shutil.rmtree(etfs_folder)

# Move the content of the "Stocks" folder to the target "stock_prices" folder
if os.path.exists(stocks_folder):
    for file_name in os.listdir(stocks_folder):
        src_file = os.path.join(stocks_folder, file_name)
        dest_file = os.path.join(target_folder, file_name)
        
        # Check if the destination file already exists
        if os.path.exists(dest_file):
            print(f"File {file_name} already exists in the target directory. Skipping.")
        else:
            shutil.move(src_file, dest_file)

    # Remove the now-empty "Stocks" folder
    shutil.rmtree(stocks_folder)

# Delete the original ZIP file if it exists
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)


File a.us.txt already exists in the target directory. Skipping.
File aa.us.txt already exists in the target directory. Skipping.
File aaap.us.txt already exists in the target directory. Skipping.
File aaba.us.txt already exists in the target directory. Skipping.
File aac.us.txt already exists in the target directory. Skipping.
File aal.us.txt already exists in the target directory. Skipping.
File aamc.us.txt already exists in the target directory. Skipping.
File aame.us.txt already exists in the target directory. Skipping.
File aan.us.txt already exists in the target directory. Skipping.
File aaoi.us.txt already exists in the target directory. Skipping.
File aaon.us.txt already exists in the target directory. Skipping.
File aap.us.txt already exists in the target directory. Skipping.
File aapl.us.txt already exists in the target directory. Skipping.
File aat.us.txt already exists in the target directory. Skipping.
File aau.us.txt already exists in the target directory. Skipping.
File a

In [9]:
print("Files in stock_prices:", os.listdir(target_folder))


Files in stock_prices: ['.gitkeep', 'a.us.txt', 'aa.us.txt', 'aaap.us.txt', 'aaba.us.txt', 'aac.us.txt', 'aal.us.txt', 'aamc.us.txt', 'aame.us.txt', 'aan.us.txt', 'aaoi.us.txt', 'aaon.us.txt', 'aap.us.txt', 'aapl.us.txt', 'aat.us.txt', 'aau.us.txt', 'aav.us.txt', 'aaww.us.txt', 'aaxn.us.txt', 'ab.us.txt', 'abac.us.txt', 'abax.us.txt', 'abb.us.txt', 'abbv.us.txt', 'abc.us.txt', 'abcb.us.txt', 'abcd.us.txt', 'abco.us.txt', 'abdc.us.txt', 'abe.us.txt', 'abeo.us.txt', 'abeow.us.txt', 'abev.us.txt', 'abg.us.txt', 'abil.us.txt', 'abio.us.txt', 'ablx.us.txt', 'abm.us.txt', 'abmd.us.txt', 'abr.us.txt', 'abrn.us.txt', 'abr_a.us.txt', 'abr_b.us.txt', 'abr_c.us.txt', 'abt.us.txt', 'abtx.us.txt', 'abus.us.txt', 'abx.us.txt', 'aby.us.txt', 'ac.us.txt', 'acad.us.txt', 'acbi.us.txt', 'acc.us.txt', 'acco.us.txt', 'accp.us.txt', 'acer.us.txt', 'acerw.us.txt', 'acet.us.txt', 'acfc.us.txt', 'acgl.us.txt', 'acglo.us.txt', 'acglp.us.txt', 'ach.us.txt', 'achc.us.txt', 'achn.us.txt', 'achv.us.txt', 'acia.us.

---

In [10]:
import os
import zipfile

# Define the project root by finding "MarketPulseAnalytics" within the directory structure
current_dir = os.getcwd()
# Navigate up to the "Repo" level, where "MarketPulseAnalytics" resides
project_root = os.path.join(current_dir, "..", "MarketPulseAnalytics")

# Change to the project root directory
os.chdir(project_root)
print("New current directory:", os.getcwd())

# Define paths relative to the project root
downloaded_folder = os.path.join("data", "raw", "insider_transactions", "downloaded")
extracted_folder = os.path.join("data", "raw", "insider_transactions")

# Ensure both the extracted folders exist
os.makedirs(extracted_folder, exist_ok=True)

# Check and list all ZIP files in the downloaded folder
if not os.path.exists(downloaded_folder):
    print(f"No folder named 'downloaded' found in {downloaded_folder}.")
else:
    zip_files = [f for f in os.listdir(downloaded_folder) if f.endswith('.zip')]
    if not zip_files:
        print("No ZIP files found in the downloaded folder.")
    else:
        # Process each ZIP file
        for zip_file in zip_files:
            zip_file_path = os.path.join(downloaded_folder, zip_file)
            
            # Create a new folder named after the ZIP file (without .zip) for extraction
            zip_folder_name = os.path.splitext(zip_file)[0]
            destination_folder = os.path.join(extracted_folder, zip_folder_name)
            os.makedirs(destination_folder, exist_ok=True)
            print(f"Extracting {zip_file} into {destination_folder}...")

            # Extract the contents of the ZIP file
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(destination_folder)

        print("All ZIP files have been extracted into their respective folders.")


New current directory: c:\Users\sawaomar\Documents\Project 5\Repo\MarketPulseAnalytics
Extracting 2014q1_form345.zip into data\raw\insider_transactions\2014q1_form345...
Extracting 2014q2_form345.zip into data\raw\insider_transactions\2014q2_form345...
Extracting 2014q3_form345.zip into data\raw\insider_transactions\2014q3_form345...
Extracting 2014q4_form345.zip into data\raw\insider_transactions\2014q4_form345...
Extracting 2015q1_form345.zip into data\raw\insider_transactions\2015q1_form345...
Extracting 2015q2_form345.zip into data\raw\insider_transactions\2015q2_form345...
Extracting 2015q3_form345.zip into data\raw\insider_transactions\2015q3_form345...
Extracting 2015q4_form345.zip into data\raw\insider_transactions\2015q4_form345...
Extracting 2016q1_form345.zip into data\raw\insider_transactions\2016q1_form345...
Extracting 2016q2_form345.zip into data\raw\insider_transactions\2016q2_form345...
Extracting 2016q3_form345.zip into data\raw\insider_transactions\2016q3_form345...


# Step 2: Insider Trading ETL

## 2a: NONDERIV_TRANS.TSV (from different quearters and years)

### read all files and concat

In [13]:
# Create a list of all the files
files = [os.path.join('..', 'data', 'raw', 'insider_transactions', f'{year}q{quarter}_form345', 'NONDERIV_TRANS.tsv')
         for year in range(2014, 2018) for quarter in range(1, 5)]
# Read all the files and store them in a list
dataframes = []
for file in files:
    if os.path.exists(file):
        try:
            temp = pd.read_csv(file, sep='\t', low_memory=False)
            dataframes.append(temp)
        except Exception as e:
            print(f'Error reading {file}: {e}')
    else:
        print(f'File {file} does not exist')
# Concatenate all DataFrames into one DataFrame
df = pd.concat(dataframes, ignore_index=True)


File ..\data\raw\insider_transactions\2014q1_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2014q2_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2014q3_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2014q4_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2015q1_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2015q2_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2015q3_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2015q4_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2016q1_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2016q2_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2016q3_form345\NONDERIV_TRANS.tsv does not exist
File ..\data\raw\insider_transactions\2016q

ValueError: No objects to concatenate

### remove unwanted columns

In [ ]:
# Drop columns that are not needed (ensure columns exist)(either columns have so many missing values or they are not needed)
columns_to_drop = ['DIRECT_INDIRECT_OWNERSHIP_FN',
                   'NATURE_OF_OWNERSHIP',
                   'NATURE_OF_OWNERSHIP_FN',
                   'VALU_OWND_FOLWNG_TRANS',
                   'VALU_OWND_FOLWNG_TRANS_FN',                   
                   'SHRS_OWND_FOLWNG_TRANS_FN',
                   'TRANS_ACQUIRED_DISP_CD_FN',
                   'TRANS_PRICEPERSHARE_FN',
                   'TRANS_SHARES_FN',
                   'TRANS_TIMELINESS_FN',
                   'EQUITY_SWAP_TRANS_CD_FN',
                   'TRANS_CODE',
                   'TRANS_FORM_TYPE',
                   'DEEMED_EXECUTION_DATE_FN',
                   'DEEMED_EXECUTION_DATE',
                   'TRANS_DATE_FN',
                   'SECURITY_TITLE_FN',
                   'SECURITY_TITLE']
# Drop columns if they exist in the DataFrame
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# Function to correct the year format
def correct_year_format(date_str):
    match = re.match(r'(\d{2}-\w{3}-00(\d{2}))', date_str)
    if match:
        corrected_year = date_str.replace('00', '20', 1)  # Replace the leading '00' with '20'
        return corrected_year
    return date_str

# Apply the function to the TRANS_DATE column
df['TRANS_DATE'] = df['TRANS_DATE'].apply(correct_year_format)



### adjust column values mapping

In [ ]:

# for column EQUITY_SWAP_INVOLVED, 0 = false, 1 = true
df['EQUITY_SWAP_INVOLVED'] = df['EQUITY_SWAP_INVOLVED'].astype(str)
print(df['EQUITY_SWAP_INVOLVED'].unique())
# Map the column values to ensure consistent True/False representation
df['EQUITY_SWAP_INVOLVED'] = df['EQUITY_SWAP_INVOLVED'].replace({
    'false': 'False',
    '0': 'False',
    '1': 'True',
    'true': 'True',
    'False': 'False',
    'True': 'True'
})
# Convert the column to boolean type
df['EQUITY_SWAP_INVOLVED'] = df['EQUITY_SWAP_INVOLVED'].map({'True': True, 'False': False})
# Print unique values to confirm conversion
print(df['EQUITY_SWAP_INVOLVED'].unique())



In [ ]:

# for column TRANS_TIMELINESS, E = early, L = late, O = on time
print(df['TRANS_TIMELINESS'].unique())
df['TRANS_TIMELINESS'] = df['TRANS_TIMELINESS'].replace(np.nan, 'O')
print(df['TRANS_TIMELINESS'].unique())


### remove rows where SHRS_OWND_FOLWING_TRANS is nan or TRANS_PRICEPERSHR is nan

In [ ]:
# using the column SHRS_OWND_FOLWNG_TRANS and TRANS_PRICEPERSHARE we remove any rows where the value is NaN for either column
df = df.dropna(subset=['SHRS_OWND_FOLWNG_TRANS', 'TRANS_PRICEPERSHARE'])
print(df)

### print the dataframe summary

In [ ]:
# TRANS_ACQUIRED_DISP_CD: A = acquired, D = disposed
# DIRECT_INDIRECT_OWNERSHIP: D = direct, I = indirect
# EQUITY_SWAP_INVOLVED: 0 = false, 1 = true

# Print DataFrame information
print(df.info())

## 2b: SUBMISSION.TSV (from different quearters and years)

### read all files and concat

In [ ]:
# Create a list of all the files
files = [os.path.join('..', 'data', 'raw', 'insider_transactions', f'{year}q{quarter}_form345', 'SUBMISSION.tsv')
         for year in range(2014, 2018) for quarter in range(1, 5)]
# Read all the files and store them in a list
dataframes = []
for file in files:
    if os.path.exists(file):
        try:
            temp = pd.read_csv(file, sep='\t', low_memory=False)
            dataframes.append(temp)
        except Exception as e:
            print(f'Error reading {file}: {e}')
    else:
        print(f'File {file} does not exist')
# Concatenate all DataFrames into one DataFrame
df2 = pd.concat(dataframes, ignore_index=True)


### coulmns to keep

In [ ]:
# We only keep columns: ACCESSION_NUMBER, FILING_DATE, PERIOD_OF_REPORT, ISSUERNAME, ISSUERTRADINGSYMBOL
columns_to_keep = ['ACCESSION_NUMBER', 'FILING_DATE', 'PERIOD_OF_REPORT', 'ISSUERNAME', 'ISSUERTRADINGSYMBOL']
# Drop columns that are not needed
df2.drop(columns=[col for col in df2.columns if col not in columns_to_keep], inplace=True)
# Print DataFrame information
print(df2.info())


In [ ]:
# the same company name should have the same trading symbol
# if 'ISSUERTRADINGSYMBOL' is nan, we look at its corresponding ISSUERNAME value. 
# if the corresponding ISSUERNAME is not nan, we can use it to find other rows of the same ISSUERNAME where ISSUERTRADINGSYMBOL is not nan and fill the nan value with the non-nan value.

#  if ISSUERNAME is nan, we can't do anything about it. we will just leave it as nan and drop rows where ISSUERTRADINGSYMBOL is nan 

# Create a mapping of ISSUERNAME to ISSUERTRADINGSYMBOL for non-NaN trading symbols
issuer_symbol_map = df2.dropna(subset=['ISSUERTRADINGSYMBOL']).set_index('ISSUERNAME')['ISSUERTRADINGSYMBOL'].to_dict()

# Apply the mapping to fill NaN values in ISSUERTRADINGSYMBOL based on ISSUERNAME
df2['ISSUERTRADINGSYMBOL'] = df2.apply(
    lambda row: issuer_symbol_map.get(row['ISSUERNAME'], row['ISSUERTRADINGSYMBOL']) 
    if pd.isna(row['ISSUERTRADINGSYMBOL']) and pd.notna(row['ISSUERNAME']) else row['ISSUERTRADINGSYMBOL'],
    axis=1
)

# Drop rows where ISSUERTRADINGSYMBOL is still NaN
df2.dropna(subset=['ISSUERTRADINGSYMBOL'], inplace=True)

# Print DataFrame info to verify changes
# FILING_DATE is when the form was filed to the commission
# TRANS_DATE is when the transaction was executed
# declaration of intent to trade or smth like that means that PERIOD_OF_REPORT can be before or same data as TRANS_DATE
# while filing date is maybe not needed for predictions, the report period can be useful.
# we can check if the report period is done before transaction date, indicating clear intent to trade . (maybe we can use delta between the two dates as a feature)

print(df2.info())



## 2c: REPORTING_OWNER.tsv (from different quearters and years)    

### read all files and concat

In [ ]:
# Create a list of all the files
files = [os.path.join('..', 'data', 'raw', 'insider_transactions', f'{year}q{quarter}_form345', 'REPORTINGOWNER.tsv')
         for year in range(2014, 2018) for quarter in range(1, 5)]
# Read all the files and store them in a list
dataframes = []
for file in files:
    if os.path.exists(file):
        try:
            temp = pd.read_csv(file, sep='\t', low_memory=False)
            dataframes.append(temp)
        except Exception as e:
            print(f'Error reading {file}: {e}')
    else:
        print(f'File {file} does not exist')
# Concatenate all DataFrames into one DataFrame
df3 = pd.concat(dataframes, ignore_index=True)



### remove unwanted columns

In [ ]:
# only keep RPTOWNER_RELATIONSHIP and ACCESSION_NUMBER
columns_to_keep = ['RPTOWNER_RELATIONSHIP', 'ACCESSION_NUMBER']
# Drop columns that are not needed
df3.drop(columns=[col for col in df3.columns if col not in columns_to_keep], inplace=True)
#drop nan RPTOWNER_RELATIONSHIP
df3.dropna(subset=['RPTOWNER_RELATIONSHIP'], inplace=True)
# Print DataFrame information
print(df3.info())

## 2d: Joined NONDERIV_TRANS.tsv, SUBMISSION.tsv, REPORTING_OWNER.tsv

In [ ]:
# df4= join df, df2, df3 on ACCESSION_NUMBER
df4 = df.merge(df2, on='ACCESSION_NUMBER').merge(df3, on='ACCESSION_NUMBER')
# Print DataFrame information
print(df4.info())
# TRANS_ACQUIRED_DISP_CD: A = acquired, D = disposed
# DIRECT_INDIRECT_OWNERSHIP: D = direct, I = indirect
# EQUITY_SWAP_INVOLVED: 0 = false, 1 = true
# for column TRANS_TIMELINESS, E = early, L = late, O = on time



# Step 3: Stock Prices ETL

## Read the files

In [ ]:
# Date,Open,High,Low,Close,Volume,OpenInt
#  above are the columns in the stock data. We can ignore the OpenInt column as it is not needed.
# the folder structure is ../data/raw/stock_data/xxx.us.txt where xxx is the stock symbol.
# before the first '.' delimiter, we have the symbol name.
# After the second '.' delimiter, we have the country name (us in this case). 
#  Therefore, the Insider Trading data's ISSUERTRADINGSYMBOL column should match the stock symbol name. The country name here seems irrelevant since in the REPORTINGOWNER.tsv file, we have the country name but that country is like the address of reporting owner (person) and not the stock.
#  Therefore, we can ignore the country name in the stock data file name and just match the symbol name.

#  let's read all files and extract the symbol name from the file name and store it in a new column called 'SYMBOL'. We will remove OpenInt column as well.





# Create a list of all the files
files = [os.path.join('..', 'data', 'raw', 'stock_prices', filename) 
         for filename in os.listdir(os.path.join('..', 'data', 'raw', 'stock_prices')) 
         if filename.endswith('.txt')]
# Read all the files and store them in a list
dataframes = []
for file in files:
    #  extract symbol name from file name which is the string before the first '.' delimiter in the file name
    symbol = os.path.basename(file).split('.')[0]
    if os.path.exists(file):
        try:
            temp = pd.read_csv(file, sep=',', low_memory=False)
            # Add a new column 'SYMBOL' with the symbol name
            temp['SYMBOL'] = symbol
            # remove OpenInt column
            temp.drop(columns=['OpenInt'], inplace=True)
            # filter dates to be from 2014 till 2017 (inclusive and all months)
            temp = temp[temp['Date'].str.startswith('2014') | temp['Date'].str.startswith('2015') | temp['Date'].str.startswith('2016') | temp['Date'].str.startswith('2017')]
            dataframes.append(temp)
        except Exception as e:
            print(f'Error reading {file}: {e}')# as an example: Error reading ..\data\raw\stock_prices\accp.us.txt: No columns to parse from file (empty data file)

    else:
        print(f'File {file} does not exist')
# Concatenate all DataFrames into one DataFrame
df5 = pd.concat(dataframes, ignore_index=True)




# Step 4: Merging Insider Trading and Stock Prices

In [ ]:
#  the stocks prices dataset has  Symbol and Date columns (Date and SYMBOL)
#  the insider trading data has the transaction date and the stock symbol name (TRANS_DATE and ISSUERTRADINGSYMBOL)TRANS_DATE HAS THE FORM 13-MAR-2014

# therefore, we can join the insider trading data with the stock prices data on the stock symbol name and the transaction date. dATE HAS THE FORM 2014-01-23    
# df4 is the insider trading data and df5 is the stock prices data

# Ensure both dataframes have symbol columns in the same case (e.g., uppercase)
df4['ISSUERTRADINGSYMBOL'] = df4['ISSUERTRADINGSYMBOL'].str.upper()
df5['SYMBOL'] = df5['SYMBOL'].str.upper()

# Convert TRANS_DATE to the same format as Date in df5
df4['TRANS_DATE'] = pd.to_datetime(df4['TRANS_DATE'], format='%d-%b-%Y').dt.strftime('%Y-%m-%d')

# Merge the insider trading data with the stock prices data on the stock symbol name and the transaction date
merged_df = pd.merge(df4, df5, left_on=['ISSUERTRADINGSYMBOL', 'TRANS_DATE'], right_on=['SYMBOL', 'Date'], how='inner')

# Print the merged DataFrame information
print(merged_df.info())


# Step 5: Saving Interim Data

In [ ]:
# We have 1,322,820 million rows for all the insider trading data files.(7,877 unique symbols)
# We have 5,442,556 rows for the stocks price data files. (7,163 unique symbols)
# merging both based on the stock symbol name and the transaction date, we have 978,647 rows. (4,450 unique symbols)
# from 2014 to 2017, we have 1,043 working business days.

# naturally, the insider trading data is less than the stock prices data as not all companies have insider trading data.
# the merged data could be useful for predicting stock prices based on insider trading data.(direct daily relationship between insider trading data and stock prices)
# but there will be many more data points in the stock prices that have no corresponding insider trading data. (indirect relationship between insider trading data and stock prices).
# in our plot, we can first plot all stocks prices and then color-code the points that have insider trading data vs those that don't have insider trading data.


#  for now, let's save the df4,  to the folder path ../data/interim/insider_transactions
#  let's save the df5 to the folder path ../data/interim/stock_prices
# let's save the merged_df to the folder path ../data/interim/merged_insider_transactions_stock_prices
# we save using paths and os packages that work on all operating systems.
#  if the folders do not exist, we create them.

# Define the folder paths
insider_transactions_path = os.path.join('..', 'data', 'interim', 'insider_transactions')
stock_prices_path = os.path.join('..', 'data', 'interim', 'stock_prices')
merged_path = os.path.join('..', 'data', 'interim', 'merged_insider_transactions_stock_prices')

# Create directories if they do not exist
os.makedirs(insider_transactions_path, exist_ok=True)
os.makedirs(stock_prices_path, exist_ok=True)
os.makedirs(merged_path, exist_ok=True)

# Save the DataFrames to the respective paths
df4.to_csv(os.path.join(insider_transactions_path, 'interim_insider_transactions.csv'), index=False)
df5.to_csv(os.path.join(stock_prices_path, 'interim_stock_prices.csv'), index=False)
merged_df.to_csv(os.path.join(merged_path, 'interim_merged_insider_transactions_stock_prices.csv'), index=False)